In [26]:
import pandas as pd
from itertools import islice

import mxnet as mx

from gluonts.dataset.common import ListDataset
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.wavenet import WaveNetEstimator
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions

from demo_utils import *


TRAIN_START, TRAIN_END = '2012-01-01 00:00:00', '2014-09-01 00:00:00'
TEST_START, TEST_END = '2014-08-25 00:00:00', '2014-09-08 00:00:00'

# df = pd.read_csv('data/LD2011_2014.txt', sep=";", index_col=0, parse_dates=True, decimal=',')
# df = df.resample('1H', label='left', closed='right').sum()[TRAIN_START:TEST_END]
# inds = (df.values != 0).mean(0) > 0.9
# df = df.iloc[:, inds]
# df.to_csv('electricity.csv', index=True)

df = pd.read_csv('electricity.csv',index_col=0,parse_dates = True)
cardinality = df.shape[1]

In [1]:
mx.random.seed(1)

# data preparation
train_data = ListDataset([{'start': TRAIN_START,
                           'target': df.loc[TRAIN_START:TRAIN_END].iloc[:, i].tolist(),
                           'feat_static_cat': [i]}
                          for i in range(cardinality)],
                         freq='1H')

test_data = ListDataset([{'start': TRAIN_START,
                          'target': df.loc[TRAIN_START:TEST_END].iloc[:, i].tolist(),
                          'feat_static_cat': [i]}
                         for i in range(cardinality)],
                        freq='1H')

# train
estimator = DeepAREstimator(freq="1H", prediction_length=24, context_length=168,
                            trainer=Trainer(epochs=20, batch_size=64),
                            use_feat_static_cat=True, cardinality=[cardinality])
# estimator = MQCNNEstimator(freq="1H", prediction_length=24, context_length=168,
#                            trainer=Trainer(epochs=20, batch_size=64), quantiles=[0.1, 0.5, 0.9])
# estimator = WaveNetEstimator(freq="1H", prediction_length=24,
#                              trainer=Trainer(epochs=20, batch_size=64), cardinality=[cardinality])

predictor = estimator.train(training_data=train_data, validation_data=test_data)

# inference
target_ind = 10
for i, (test_entry, forecast) in enumerate(islice(zip(test_data, predictor.predict(test_data)), target_ind, target_ind+1)):
    pred_df = plot_prediction(test_entry, forecast)

In [22]:
df.loc[:3]

,Unnamed: 0,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,...,MT_360,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369
0,2012-01-01 00:00:00,16.497462,91.038407,309.296264,552.845528,289.024390,1392.857143,34.482759,1117.845118,291.958042,...,876.096491,482.512491,103900.0,6042.194093,6636.363636,62.581486,51.492101,2096.575944,243.739566,2854.105572
1,2012-01-01 01:00:00,19.035533,92.460882,309.296264,548.780488,280.487805,1357.142857,27.699265,1104.377104,269.230769,...,808.114035,451.106353,95500.0,4388.185654,6613.636364,63.885267,42.129901,2200.175593,245.409015,2823.313783
2,2012-01-01 02:00:00,16.497462,91.749644,309.296264,546.747967,265.853659,1145.833333,26.568683,959.595960,255.244755,...,813.596491,410.421128,96600.0,4278.481013,6568.181818,63.885267,38.619075,1996.488147,203.672788,2756.598240
3,2012-01-01 03:00:00,19.035533,88.193457,309.296264,408.536585,201.219512,767.857143,19.219898,801.346801,166.083916,...,797.149123,259.814418,93400.0,4168.776371,6318.181818,73.011734,38.619075,1320.456541,188.647746,2724.340176


In [11]:
WaveNetEstimator?

Init signature: WaveNetEstimator(freq:str, prediction_length:int, trainer:gluonts.trainer._base.Trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=200, hybridize=False, init="xavier", learning_rate=0.01, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=50, patience=10, weight_decay=1e-08), cardinality:List[int]=[1], seasonality:Union[int, NoneType]=None, embedding_dimension:int=5, num_bins:int=1024, hybridize_prediction_net:bool=False, n_residue=24, n_skip=32, dilation_depth:Union[int, NoneType]=None, n_stacks:int=1, train_window_length:Union[int, NoneType]=None, temperature:float=1.0, act_type:str='elu', num_parallel_samples:int=200) -> None
Docstring:     
Model with Wavenet architecture and quantized target.

Parameters
----------
freq
    Frequency of the data to train on and predict
prediction_length
    Length of the prediction horizon
trainer
    Trainer object to be used (default: Trainer())
cardinality
  